In [1]:
# import pandas (library for dataframes)
import pandas as pd
pd.set_option('display.max_columns', None)
# import listdir (library to list directories)
from os import listdir
import numpy as np
import datetime
import xlsxwriter

In [2]:
def breathing_calculations(file_name, file_suffix, file_location):

    # Check if file exists in file location
    file_exists = False
    for dir_file in listdir(file_location):
        if dir_file == file_name + file_suffix:
            file_exists = True
    
    if not file_exists:
        print("File: " + file_name + file_suffix + " does not exist in " + file_location)
        raise Exception("File: " + file_name + file_suffix + " does not exist in " + file_location)

    # Read in data from file into dataframe
    data = None
    column_names = ["row_number","cpu_date","cpu_time","site_time","period_time","Action","title",
                "description","Blank", "Breath_Num","first_beat_id","last_beat_id", "Ti","Te",
                "PIF","PEF", "TV", "EV","RT","MV", "f", "EIP", "EEP", "Penh", "EF50", "RH", 
                "Tbox", "Tbody", "Patm", "VCF", "AV", "Sr", "n"]
    with open(file_location + file_name + file_suffix, encoding='unicode_escape') as f:
        data = pd.read_csv(f, header=None, names=column_names, skip_blank_lines=True, na_filter=True, dtype='string', skipinitialspace=True, sep='\t')

    # row numbers when Action column is period-start
    period_start_rows = data[data["Action"] == "period-start"].index

    # start the data at the row of the first period-start Action
    first_period_start = int(period_start_rows[0])
    print(first_period_start)

    # row numbers when Action column is period-stop
    period_stop_rows = data[data["Action"] == "period-stop "].index

    # end the data at the row of the last period-stop Action
    last_period_stop = int(period_stop_rows[-1])
    print(last_period_stop)

    # change data to be from first period-start to last period-stop
    data = data.iloc[first_period_start:last_period_stop+1]

    # Drop unnecessary columns/rows
    data = data.drop(["first_beat_id","last_beat_id"], axis="columns")
    data = data.drop("Blank", axis="columns")
    data = data.drop(["row_number", "cpu_date", "cpu_time"], axis="columns")

    analyzer_row_numbers = data[data["Action"] == "analyzer-tuning"].index
    data = data.drop(analyzer_row_numbers)

    # convert necessary columns to float types
    num_cols = len(data.columns)
    list_cols_to_convert_to_float = list(range(6, num_cols))
    for i in list_cols_to_convert_to_float:
        data.iloc[:, i] = data.iloc[:, i].map(float, na_action='ignore')

    # Reset index to start from 0
    data = data.reset_index(drop=True)

    # Period start and period stop rows change after reset_index, so reassign variables
    period_start_rows = data[data["Action"] == "period-start"].index
    period_stop_rows = data[data["Action"] == "period-stop "].index

    # Number of Breaths per condition
    num_breaths_per_condition = period_stop_rows - period_start_rows - 1
    print(num_breaths_per_condition)

    # Get title condition for each period-start Action
    conditions = list(data[data["Action"] == "period-start"]["title"].values)
    print(conditions)

    # Replace Ramp if exists
    if "Hypoxia" in conditions:
        hypoxia_index = conditions.index("Hypoxia")
        # Change Ramp to Ramp Up After Hypoxia (if exists)
        if conditions[hypoxia_index + 1] == "Ramp":
            conditions[hypoxia_index + 1] = "Ramp Up"

        # Change Ramp to Ramp Down Before Hypoxia (if exists)
        if conditions[hypoxia_index - 1] == "Ramp":
            conditions[hypoxia_index - 1] = "Ramp Down"

    # Keep all conditions not equal (ne) to Ramp
    # If ramp is removed, be sure to remove ramp data as well
    # conditions = list(filter("Ramp".__ne__, conditions))
    # print(conditions)

    # Loop through number of conditions to average data for each condition
    averaged_data = pd.DataFrame()
    for i in range(len(conditions)):
        condition_start_row = period_start_rows[i] + 1
        condition_stop_row = period_stop_rows[i] - 1
        condition_rows_data = data.iloc[condition_start_row:condition_stop_row, 6:]
        filtered_TV = condition_rows_data[condition_rows_data["TV"] < 0.8]
        one_averaged_data = filtered_TV.mean()
        averaged_data = averaged_data.append(one_averaged_data, ignore_index=True)

    # Join Breaths Dataframe with Averages Dataframe
    breaths_dataframe = pd.DataFrame(num_breaths_per_condition, columns=["Breath_Count"])
    averaged_data = breaths_dataframe.join(averaged_data)

    # Differentiate Conditions using Dictionary to keep track of number of duplicates
    numbered_conditions = []
    number_dict = {}
    for condition in conditions:
        if condition in number_dict:
            number_dict[condition] += 1
            numbered_conditions.append(condition + " " + str(number_dict[condition]))
        else:
            number_dict[condition] = 1
            numbered_conditions.append(condition)

    # Join Conditions Dataframe with Averages Dataframe
    conditions_dataframe = pd.DataFrame(numbered_conditions, columns=["Conditions"])
    averaged_data = conditions_dataframe.join(averaged_data)

    # Condition reduction
    averaged_data = condition_reduction("Baseline", "Post Injection", "Base_PI_Red", averaged_data)

    # Duplicate file_directory row information averaged_data index # of times to join together
    file_dir_row = file_dir_data[file_dir_data["FileName"] == file_name]
    file_dir_row = file_dir_row.append([file_dir_row]*(len(averaged_data.index) - 1),ignore_index=True)

    # Join file_directory mouse info with averaged_data
    mouse_averaged_data = file_dir_row.join(averaged_data)
    return mouse_averaged_data

In [3]:
def condition_mean(cond1, cond2, row_name, averaged_data):
    if cond1 in averaged_data["Conditions"].values and cond2 in averaged_data["Conditions"].values:
        cond1_data = averaged_data[averaged_data["Conditions"] == cond1].iloc[:, 1:].values[0]
        cond2_data = averaged_data[averaged_data["Conditions"] == cond2].iloc[:, 1:].values[0]
        mean = [(cond1_i + cond2_i) / 2 for cond1_i, cond2_i in zip(cond1_data, cond2_data)]
        mean.insert(0, row_name)
        averaged_data.loc[len(averaged_data.index)] = mean
        return averaged_data

In [4]:
def condition_reduction(cond1, cond2, row_name, averaged_data):
    if cond1 in averaged_data["Conditions"].values and cond2 in averaged_data["Conditions"].values:
        # Get values of condition 1 data and condition 2 data as lists
        cond1_data = averaged_data[averaged_data["Conditions"] == cond1].iloc[:, 1:].values[0]
        cond2_data = averaged_data[averaged_data["Conditions"] == cond2].iloc[:, 1:].values[0]
        reduction = list((np.subtract(cond2_data, cond1_data) / cond1_data) * 100)
        reduction.insert(0, row_name)
        averaged_data.loc[len(averaged_data.index)] = reduction
        return averaged_data

In [5]:
# open file directory and save data to dataframe
with open('inputs/injection_opioids/___FILE DIRECTORY___.csv', encoding='unicode_escape') as f:
    file_dir_data = pd.read_csv(f, header=0, skip_blank_lines=True, na_filter=True, dtype='string', skipinitialspace=True, sep=',')
file_dir_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick
0,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>
1,220710_chamber1_ajc7022_m02bl_08212022_68135,1,1,DA,07/10/2022,<NA>,ajc7022,m,bl,2,filler,filler,filler,filler,filler,24.8,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>
2,220710_chamber2_ajc6802_m02bl_08212022_66318,2,1,DA,07/10/2022,<NA>,ajc6802,m,bl,2,filler,filler,filler,filler,filler,28.2,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>
3,220710_chamber2_ajc7023_m30bl_08212022_68135,2,1,DA,07/10/2022,<NA>,ajc7023,m,bl,30,filler,filler,filler,filler,filler,23.9,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,Outlier
4,220731_chamber1_ajc6797_m30bl_08212022_68251,1,1,DA,07/31/2022,<NA>,ajc6797,m,bl,30,filler,filler,filler,filler,filler,28.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>
5,220731_chamber1_ajc6801_m20bl_08212022_68333,1,2,DA,07/31/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.4,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>
6,220731_chamber2_ajc6796_m20bl_08212022_68333,2,1,DA,07/31/2022,<NA>,ajc6796,m,bl,20,filler,filler,filler,filler,filler,28.6,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>
7,220731_chamber2_ajc6798_m00bl_08212022_65563,2,1,DA,07/31/2022,<NA>,ajc6798,m,bl,0,filler,filler,filler,filler,filler,29.4,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>
8,220807_chamber1_ajc6798_m00bl_08212022_68456,1,2,DA,08/07/2022,<NA>,ajc6798,m,bl,0,filler,filler,filler,filler,filler,29.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>

In [6]:
# Convert weight column to float for calculations
file_dir_data["Weight"] = file_dir_data["Weight"].map(float, na_action=None)

# Convert Date_Tested column to datetime for calculations
file_dir_data["Date_Tested"] = file_dir_data["Date_Tested"].map(lambda date: datetime.datetime.strptime(date, "%Y-%m-%d"))

# Convert DOB column to datetime for calculations
# file_dir_data["DOB"] = file_dir_data["DOB"].map(lambda date: datetime.datetime.strptime(date, "%Y-%m-%d"))

ValueError: time data '07/10/2022' does not match format '%Y-%m-%d'

In [7]:
# Add Age column
# file_dir_data["Age"] = file_dir_data["Date_Tested"] - file_dir_data["DOB"]
file_dir_data["Age"] = fillna()

NameError: name 'fillna' is not defined

In [8]:
analysis_names = ["Conditions","Breath_Count", "Ti","Te","PIF","PEF", 
                "TV", "EV","RT","MV", "f", "EIP", "EEP", "Penh", "EF50", 
                "RH", "Tbox","Tbody", "Patm","VCF", "AV", "Sr","n"]

master_data_columns = list(file_dir_data.columns.values) + analysis_names
master_data_columns

['FileName',
 'Chamber',
 'Test_Number',
 'Tester',
 'Date_Tested',
 'DOB',
 'Mouse_ID',
 'Sex',
 'Coat_Color',
 'Earclip',
 'Mouse_Strain',
 'Gene',
 'Genotype',
 'Gene2',
 'Genotype2',
 'Weight',
 'Include',
 'Baseline_Filter_Less_Strict',
 'Baseline_Filter_More_Strict',
 'Experiment_Type',
 'Anesthesia',
 'Anesthesia_Dose',
 'Flow_Threshold',
 'Injection_Type',
 'Injection_Success',
 'Injection_Attempts',
 'Injection1',
 'Injection1_Stock_Conc',
 'Injection1_Stock_Conc_Units',
 'Injection1_Stock_Prep_Date',
 'Injection1_Solvent',
 'Injection1_Solvent_Percent',
 'Injection1_Dosage',
 'Injection1_Dosage_Units',
 'Injection1_volume_ml',
 'Injection2',
 'Injection2_Stock_Conc',
 'Injection2_Stock_Conc_Units',
 'Injection2_Stock_Prep_Date',
 'Injection2_Solvent',
 'Injection2_Solvent_Percent',
 'Injection2_Dosage',
 'Injection2_Dosage_Units',
 'Injection2_volume_ml',
 'Protocol',
 'Raw_Protocol',
 'Notes',
 'Notes for Nick',
 'Conditions',
 'Breath_Count',
 'Ti',
 'Te',
 'PIF',
 'PEF',
 

In [9]:
# Create empty master dataframe
master_data = pd.DataFrame(columns=master_data_columns)
master_data
# TODO: Should I do something with this?
# master_data$Date_Tested <- as.Date(master_data$Date_Tested,"%Y-%m-%d")
# master_data$DOB <- as.Date(master_data$DOB, "%m/%d/%y")

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Conditions,Breath_Count,Ti,Te,PIF,PEF,TV,EV,RT,MV,f,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n


In [10]:
# Retrieve file list from file directory
file_list = file_dir_data["FileName"]
file_list

0       220710_chamber1_ajc6801_m20bl_08212022_66318
1       220710_chamber1_ajc7022_m02bl_08212022_68135
2       220710_chamber2_ajc6802_m02bl_08212022_66318
3       220710_chamber2_ajc7023_m30bl_08212022_68135
4       220731_chamber1_ajc6797_m30bl_08212022_68251
5       220731_chamber1_ajc6801_m20bl_08212022_68333
6       220731_chamber2_ajc6796_m20bl_08212022_68333
7       220731_chamber2_ajc6798_m00bl_08212022_65563
8       220807_chamber1_ajc6798_m00bl_08212022_68456
9       220807_chamber1_ajc7022_m02bl_08212022_68574
10      220807_chamber2_ajc6797_m30bl_08212022_68574
11      220807_chamber2_ajc6802_m02bl_08212022_68456
12      220821_chamber1_ajc6801_m20bl_08212022_83226
13      220821_chamber1_ajc7022_m02bl_08212022_83344
14      220821_chamber2_ajc6796_m20bl_08212022_83226
15      220821_chamber2_ajc6797_m30bl_08212022_83344
16      220911_chamber1_ajc6802_m02bl_09112022_81637
17      220911_chamber2_ajc6798_m00bl_09112022_81637
18      220918_chamber1_ajc6796_m20bl_09182022

In [11]:
# Apply breathing_calculations to each file
file_location = "inputs/injection_opioids/"
file_suffix = ".rf_1.iox.txt"
for file_name in file_list:
    master_data = master_data.append(breathing_calculations(file_name, file_suffix, file_location))
master_data

146
7898
Int64Index([1518, 85, 760, 3, 247, 0, 39, 2377, 2, 54, 2440], dtype='int64')
['Baseline', 'Ramp', 'Hypoxia', 'Ramp', 'Normoxia', 'Injection 1', 'Ramp', 'Post Injection', 'Injection 2', 'Ramp', 'Recovery']
48
6804
Int64Index([1335, 109, 863, 24, 984, 0, 34, 1758, 4, 62, 1412], dtype='int64')
['Baseline', 'Ramp', 'Hypoxia', 'Ramp', 'Normoxia', 'Injection 1', 'Ramp', 'Post Injection', 'Injection 2', 'Ramp', 'Recovery']
186
6460
Int64Index([293, 133, 700, 5, 500, 2, 57, 1895, 1, 133, 2337], dtype='int64')
['Baseline', 'Ramp', 'Hypoxia', 'Ramp', 'Normoxia', 'Injection 1', 'Ramp', 'Post Injection', 'Injection 2', 'Ramp', 'Recovery']
44
4108
Int64Index([230, 45, 614, 36, 490, 1, 24, 1443, 6, 56, 903], dtype='int64')
['Baseline', 'Ramp', 'Hypoxia', 'Ramp', 'Normoxia', 'Injection 1', 'Ramp', 'Post Injection', 'Injection 2', 'Ramp', 'Recovery']
50
3606
Int64Index([239, 21, 727, 0, 221, 0, 32, 1090, 6, 94, 939], dtype='int64')
['Baseline', 'Ramp', 'Hypoxia', 'Ramp', 'Normoxia', 'Injectio

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Conditions,Breath_Count,Ti,Te,PIF,PEF,TV,EV,RT,MV,f,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n
0,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Baseline,1518.0,99.847323,339.696629,4.507607,2.647231,0.269114,0.266252,186.152016,39.470588,144.932584,1.135492,52.212822,0.530899,1.070476,51.719762,20.248810,37.5,1009.000000,9.556213,210.939855,90.785856,1.0
1,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Ramp Down,85.0,73.180723,155.457831,6.521205,3.711807,0.339277,0.339518,115.132530,90.265060,266.337349,1.192771,11.192771,0.206506,2.819639,47.806024,20.454699,37.5,1009.000000,9.550000,425.590361,78.951807,1.0
2,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Hypoxia,760.0,91.259309,239.847074,5.022128,3.876995,0.295146,0.294987,180.414894,54.613032,184.626330,1.003989,15.067819,0.274082,1.148976,51.682979,20.550864,37.5,1009.000000,9.663098,559.073138,97.559840,1.0
3,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Ramp Up,3.0,82.000000,206.000000,4.505000,3.800000,0.230000,0.195000,147.500000,47.500000,211.000000,1.000000,9.500000,0.325000,0.580000,54.250000,20.565000,37.5,1009.000000,9.715000,669.500000,10.000000,1.0
4,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Normoxia,247.0,86.176230,304.270492,4.866639,2.556680,0.255205,0.245861,166.143443,41.459016,162.282787,1.155738,48.258197,0.483566,1.103607,55.274590,20.530943,37.5,1009.000000,9.725000,703.168033,79.176230,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [12]:
# Normalize MV, TV by weight
master_data["MV/g"] = master_data["MV"] / master_data["Weight"]
master_data["TV/g"] = master_data["TV"] / master_data["Weight"]
master_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Conditions,Breath_Count,Ti,Te,PIF,PEF,TV,EV,RT,MV,f,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n,MV/g,TV/g
0,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Baseline,1518.0,99.847323,339.696629,4.507607,2.647231,0.269114,0.266252,186.152016,39.470588,144.932584,1.135492,52.212822,0.530899,1.070476,51.719762,20.248810,37.5,1009.000000,9.556213,210.939855,90.785856,1.0,1.435294,0.009786
1,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Ramp Down,85.0,73.180723,155.457831,6.521205,3.711807,0.339277,0.339518,115.132530,90.265060,266.337349,1.192771,11.192771,0.206506,2.819639,47.806024,20.454699,37.5,1009.000000,9.550000,425.590361,78.951807,1.0,3.282366,0.012337
2,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Hypoxia,760.0,91.259309,239.847074,5.022128,3.876995,0.295146,0.294987,180.414894,54.613032,184.626330,1.003989,15.067819,0.274082,1.148976,51.682979,20.550864,37.5,1009.000000,9.663098,559.073138,97.559840,1.0,1.985928,0.010733
3,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Ramp Up,3.0,82.000000,206.000000,4.505000,3.800000,0.230000,0.195000,147.500000,47.500000,211.000000,1.000000,9.500000,0.325000,0.580000,54.250000,20.565000,37.5,1009.000000,9.715000,669.500000,10.000000,1.0,1.727273,0.008364
4,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Normoxia,247.0,86.176230,304.270492,4.866639,2.556680,0.255205,0.245861,166.143443,41.459016,162.282787,1.155738,48.258197,0.483566,1.103607,55.274590,20.530943,37.5,1009.000000,9.725000,703.168033,79.176230,1.0,1.507601,0.009280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [13]:
# Prioritize MV, TV, f columns by rearranging column order
curr_column_order = master_data.columns.tolist()
breath_count_index = curr_column_order.index("Breath_Count")
curr_column_order.remove("MV")
curr_column_order.remove("TV")
curr_column_order.remove("f")
curr_column_order.remove("MV/g")
curr_column_order.remove("TV/g")
curr_column_order.insert(breath_count_index + 1, "MV")
curr_column_order.insert(breath_count_index + 2, "TV")
curr_column_order.insert(breath_count_index + 3, "f")
curr_column_order.insert(breath_count_index + 4, "MV/g")
curr_column_order.insert(breath_count_index + 5, "TV/g")
master_data = master_data[curr_column_order]
master_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Conditions,Breath_Count,MV,TV,f,MV/g,TV/g,Ti,Te,PIF,PEF,EV,RT,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n
0,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Baseline,1518.0,39.470588,0.269114,144.932584,1.435294,0.009786,99.847323,339.696629,4.507607,2.647231,0.266252,186.152016,1.135492,52.212822,0.530899,1.070476,51.719762,20.248810,37.5,1009.000000,9.556213,210.939855,90.785856,1.0
1,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Ramp Down,85.0,90.265060,0.339277,266.337349,3.282366,0.012337,73.180723,155.457831,6.521205,3.711807,0.339518,115.132530,1.192771,11.192771,0.206506,2.819639,47.806024,20.454699,37.5,1009.000000,9.550000,425.590361,78.951807,1.0
2,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Hypoxia,760.0,54.613032,0.295146,184.626330,1.985928,0.010733,91.259309,239.847074,5.022128,3.876995,0.294987,180.414894,1.003989,15.067819,0.274082,1.148976,51.682979,20.550864,37.5,1009.000000,9.663098,559.073138,97.559840,1.0
3,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Ramp Up,3.0,47.500000,0.230000,211.000000,1.727273,0.008364,82.000000,206.000000,4.505000,3.800000,0.195000,147.500000,1.000000,9.500000,0.325000,0.580000,54.250000,20.565000,37.5,1009.000000,9.715000,669.500000,10.000000,1.0
4,220710_chamber1_ajc6801_m20bl_08212022_66318,1,1,DA,07/10/2022,<NA>,ajc6801,m,bl,20,filler,filler,filler,filler,filler,27.5,TRUE,<NA>,<NA>,<NA>,No,<NA>,0,IP,TRUE,<NA>,Fentanyl,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,Naloxone,<NA>,<NA>,<NA>,<NA>,<NA>,1,mg/kg,<NA>,"Baseline, Hypoxia, Normoxia, Exclude, Post Inj...","Baseline, Hypoxia, Normoxia, Injection 1, Post...",<NA>,<NA>,Normoxia,247.0,41.459016,0.255205,162.282787,1.507601,0.009280,86.176230,304.270492,4.866639,2.556680,0.245861,166.143443,1.155738,48.258197,0.483566,1.103607,55.274590,20.530943,37.5,1009.000000,9.725000,703.168033,79.176230,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [14]:
# Normalize MV and TV columns for Hypoxia/Normoxia
# TODO: WHAT TO DO WITH THIS???
# normalized = pd.DataFrame()
# normalized["MV/g"] = (master_data[master_data["Conditions"] == "Hypoxia"]["MV/g"] - master_data[master_data["Conditions"] == "Normoxia"]["MV/g"]) / master_data[master_data["Conditions"] == "Hypoxia"]["MV"]
# normalized["TV/g"] = (master_data[master_data["Conditions"] == "Hypoxia"]["TV/g"] - master_data[master_data["Conditions"] == "Normoxia"]["TV/g"]) / master_data[master_data["Conditions"] == "Hypoxia"]["TV"]
# normalized

In [15]:
# Make excel sheet easier to read by sorting by Mouse_ID and then Test_Number
master_data = master_data.sort_values(by=['Mouse_ID', 'Test_Number'])

In [ ]:
# Save master data to csv, include options?
master_data.to_csv()

',FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Include,Baseline_Filter_Less_Strict,Baseline_Filter_More_Strict,Experiment_Type,Anesthesia,Anesthesia_Dose,Flow_Threshold,Injection_Type,Injection_Success,Injection_Attempts,Injection1,Injection1_Stock_Conc,Injection1_Stock_Conc_Units,Injection1_Stock_Prep_Date,Injection1_Solvent,Injection1_Solvent_Percent,Injection1_Dosage,Injection1_Dosage_Units,Injection1_volume_ml,Injection2,Injection2_Stock_Conc,Injection2_Stock_Conc_Units,Injection2_Stock_Prep_Date,Injection2_Solvent,Injection2_Solvent_Percent,Injection2_Dosage,Injection2_Dosage_Units,Injection2_volume_ml,Protocol,Raw_Protocol,Notes,Notes for Nick,Age,Conditions,Breath_Count,MV,TV,f,MV/g,TV/g,Ti,Te, PIF,PEF,EV,RT,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n\n'

In [16]:
# Fuunction to highlight every Test_Number == 1
def highlight_every_number_1(s):
    df = s.copy()
    #set by condition
    mask = df['Test_Number'] == "1"
    df.loc[mask, :] = 'background-color: LightGreen'
    df.loc[~mask,:] = 'color: SlateGrey'
    return df   

In [17]:
# Separate master_data based on conditions and write all to one excel sheet
condition_list = list(set(master_data["Conditions"].values))
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
for condition in condition_list:
    condition_dataframe = master_data[master_data["Conditions"] == condition]
    condition_dataframe = condition_dataframe.reset_index(drop=True)
    condition_dataframe.style.apply(highlight_every_number_1, axis=None).to_excel(writer, sheet_name=condition)
writer.close()

In [ ]:
# Writing to Excel file
master_data.to_excel(excel_writer="output.xlsx")